## Install Dependencies and Initial Plan


1. Model Setup (Ollama + DeepSeek)

2. Data Preparation (Knowledge Source)

3. Chunk & Embed Data

4. Build Vector DB

5. Query Pipeline (RAG Flow)

6. Streamlit Setup (UI)

In [2]:
!pip install langchain langchain-community langchain-core



  Using cached langchain-0.3.23-py3-none-any.whl (1.0 MB)
  Using cached langchain_core-0.3.51-py3-none-any.whl (423 kB)
  Using cached langsmith-0.3.31-py3-none-any.whl (358 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
  Using cached sqlalchemy-2.0.40-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
  Using cached greenlet-3.1.1-cp310-cp310-win_amd64.whl (298 kB)


You should consider upgrading via the 'F:\Chatbot\chatbot_env\Scripts\python.exe -m pip install --upgrade pip' command.


## Model Setup (Ollama + DeepSeek)

In [6]:
import ollama

# Function to get response from Ollama DeepSeek 1.5B
def ollama_response(query: str) -> str:
    response = ollama.chat(model="deepseek-1.5b", messages=[{"role": "user", "content": query}])
    return response['text']

# Testing the setup with a simple query
print(ollama_response("Hello, how are you?"))


ResponseError: model "deepseek-1.5b" not found, try pulling it first (status code: 404)

In [12]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

# Instantiate the model
llm = Ollama(model="deepseek-r1:1.5b")

# Example prompt setup
prompt = ChatPromptTemplate.from_template("Tell me a joke about AI.")

# Run the model with the prompt
response = llm.invoke(prompt.format())

print(response)


<think>
Okay, so I need to come up with a joke that's specifically about AI. Hmm, where do I start? Well, the user already gave a joke about the future of machines, which seems like a good foundation. Maybe I can build on that idea or find something new.

First, let me think about what's funny when you're talking about AI. It often involves comparing it to other technologies, maybe even the human brain. But wait, I should remember that AI isn't exactly human, but people often joke about its capabilities and how it could surpass human abilities in certain areas.

Maybe I can take a classic pun or an old joke and twist it a bit to include some tech jargon related to AI. Let me recall some common phrases with AI. Oh, "I'll never know what you've been through" comes to mind. It's about not having personal experiences but talking about them hypothetically.

But how does that tie into the joke? Well, perhaps comparing that line to a machine. The user did a good job there. Maybe I can add som

## Data Preparation (Knowledge Source)

In [19]:
#Create .txt, .pdf, or .md file (example: docs/my_faqs.txt)

#Keep clean, structured knowledge (questions, answers, docs)

import os

def load_documents_from_folder(folder_path: str):
    documents = []

    # Ensure folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Check for .txt files
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    # If no .txt file exists, create one
    if not txt_files:
        default_path = os.path.join(folder_path, "knowledge.txt")
        with open(default_path, 'w', encoding='utf-8') as f:
            f.write("This is a default knowledge base. Please add more .txt files.")
        txt_files.append("knowledge.txt")

    # Load all .txt files
    for filename in txt_files:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            documents.append(file.read())

    return documents

# Example usage
folder_path = "./knowledge_source"
documents = load_documents_from_folder(folder_path)
print(documents[:1])  # print the first document






['This is a default knowledge base. Please add more .txt files.']


## Chunk & Embed Data

In [26]:
#Use LangChain to:

#Load your documents

#Split into smaller chunks

#Embed with HuggingFaceEmbeddings

#Store into Chroma vector DB


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Use a small, fast model for local embedding
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Chunking function
def chunk_documents(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for doc in documents:
        chunks.extend(text_splitter.split_text(doc))
    return chunks

# Load knowledge from file (assuming knowledge.txt exists)
def load_knowledge_file():
    file_path = "knowledge_source/knowledge.txt"
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
            print("✅ knowledge.txt loaded successfully.")
            return content
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return ""
    except Exception as e:
        print(f"⚠️ Error reading file: {e}")
        return ""

# Load knowledge base
knowledge_text = load_knowledge_file()

# If knowledge is loaded, chunk it
if knowledge_text:
    documents = [knowledge_text]  # Wrap the text into a list to pass to chunk_documents
    chunks = chunk_documents(documents)
    print(f"First chunk: {chunks[:1]}")  # Show the first chunk
else:
    print("⚠️ No knowledge to chunk.")




✅ knowledge.txt loaded successfully.
First chunk: ['Welcome to **Upskill** — your gateway to a future in Software Quality Assurance (SQA)!\n\n🚀 **Why Choose Upskill?**\nAt Upskill, we don’t just teach — we transform beginners into QA professionals ready for the global tech industry. Our SQA course is crafted by industry experts to meet real-world needs.\n\n---']


## Build Vector DB



In [28]:
import chromadb
from langchain.vectorstores import Chroma

# Initialize Chroma client
client = chromadb.Client()

# Create or load the collection for storing the vectors
persist_directory = "./chromadb_persist"
if not os.path.exists(persist_directory):
    os.mkdir(persist_directory)

# Create collection in Chroma
collection = client.create_collection(name="chatbot_collection")

# Create the Chroma vector store
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)

# Add chunks to Chroma DB
vectorstore.add_texts(chunks)
print("Chunks added to vector DB!")



# working on this to find solution

C:\Users\Faisal\AppData\Local\Temp\ipykernel_3656\2869819951.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)


Chunks added to vector DB!


## Query Pipeline (RAG Flow)



In [52]:
#Accept user question

#Convert to embedding

#Search top-k similar chunks from Chroma

#Concatenate chunks + question

#Send to OllamaLLM via LangChain

#Get DeepSeek response



from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Replace with your desired model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use the pipeline to create a conversational model
huggingface_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Initialize HuggingFacePipeline directly without BaseChatModel
llm = HuggingFacePipeline(pipeline=huggingface_pipeline)

# Initialize the retrieval QA system with the Hugging Face model
qa_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

# Function to process query using RAG flow
def query_pipeline(query: str) -> str:
    # Initialize an empty chat history
    chat_history = []
    
    # Create a dictionary with the 'question' and 'chat_history' keys
    input_data = {"question": query, "chat_history": chat_history}
    
    # Run the query through the QA chain
    response = qa_chain.run(input_data)
    return response

# Test the query pipeline
print(query_pipeline("Tell me about Upskill."))





If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Welcome to **Upskill** — your gateway to a future in Software Quality Assurance (SQA)!

🚀 **Why Choose Upskill?**
At Upskill, we don’t just teach — we transform beginners into QA professionals ready for the global tech industry. Our SQA course is crafted by industry experts to meet real-world needs.

---

---

🎯 **Why SQA is a Great Career Path?**
- High demand globally  
- Great salary growth  
- Opportunities to work in agile teams  
- A critical role in product quality & user satisfaction  

---

📞 **Contact Upskill Today**
- Website: www.upskill.com  
- WhatsApp: +8801XXXXXXXXX  
- Email: info@upskill.com

🧠 Learn QA. Get Certified. Launch Your Career — with Upskill.

---

💡 **Who is this for?**
- Fresh graduates looking to enter tech  
- Career switchers from non-IT backgrounds  
- Professionals seeking QA upskilling  


In [44]:
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain

# === Load Ollama Model ===
# Make sure your model (like deepseek:1.5b) is running via `ollama run deepseek`
llm = Ollama(model="deepseek-r1:1.5b")

# === Build Retrieval-Augmented Generation Chain ===
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()  # Make sure vectorstore is initialized with your docs
)

# === Query Function with Chat History ===
chat_history = []

def query_pipeline(query: str) -> str:
    global chat_history
    response = qa_chain.invoke({
        "question": query,
        "chat_history": chat_history
    })
    # Update chat history for conversational memory
    chat_history.append((query, response["answer"]))
    return response["answer"]


In [45]:
print(query_pipeline("Tell me about Upskill"))



<think>
Alright, I'm trying to figure out what "Upskill" is based on the provided context. Let's go through each part of the information given step by step.

First, there's a welcome message that introduces Upskill as a gateway to software quality assurance (SQA). That tells me it's an educational or training platform focused on helping people gain expertise in SQA fields like testing and software development.

Next, under "Why Choose Upskill?", it explains that the course is crafted by industry experts to meet real-world needs. This suggests that Upskill's content is practical and relevant to actual job scenarios. They emphasize transforming beginners into QA professionals ready for the tech industry, which implies that they're preparing individuals to work in a professional environment.

The "Why SQA is a Great Career Path?" section highlights high demand globally, salary growth, agile team opportunities, and critical role in product quality and user satisfaction. This shows that Ups

## Streamlit Setup (UI)